In [1]:
import pandas as pd

In [2]:
resource_file_path = "Resources/purchase_data.csv"
# Check the encoding
with open(resource_file_path) as f:
    print(f)

<_io.TextIOWrapper name='Resources/purchase_data.csv' mode='r' encoding='UTF-8'>


In [3]:
df = pd.read_csv(resource_file_path)
# Preview data
df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [4]:
# Check if data needs to be cleaned
df.count()

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

### Player Count
- Display the total number of players

In [5]:
# Count total number of players
total_players = len(df.groupby("SN"))
print(f"Total Players: \n{total_players}")

Total Players: 
576


### Purchasing Analysis (Total)
- Run basic calculations to obtain number of unique items, average price, etc.
- Create a summary data frame to hold the results
- Optional: give the displayed data cleaner formatting
- Display the summary data frame

In [6]:
# Number of Unique Items
unique_items_number = len(df["Item Name"].unique())

# Average Purchase Price
average_purchase_price = round(df["Price"].mean(),2)

# Total Number of Purchases
total_number_purchases = len(df["Purchase ID"])

# Total Revenue
total_revenue = df["Price"].sum()

purchasing_analysis = pd.DataFrame({
    "Number of Unique Items":[unique_items_number],
    "Average Price":[f"${average_purchase_price}"],
    "Number of Purchases":[total_number_purchases],
    "Total Revenue":[f"${total_revenue}"]
})
purchasing_analysis

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


### Gender Demographics
- Percentage and Count of Male Players
- Percentage and Count of Female Players
- Percentage and Count of Other / Non-Disclosed

In [7]:
gndr = df["Gender"].unique().sort()
print(gndr)

['Male' 'Other / Non-Disclosed' 'Female']


In [414]:
gender = ["Female", "Male", "Other / Non-Disclosed"]

# Create function to count players by gender, argument specifies the gender
def count_players_by_gender(val):
    if val in [gender[0], gender[1]]:
        gender_df = df.loc[df['Gender'] == val]
    else: gender_df = df.loc[(df['Gender'] != gender[0])&(df['Gender'] != gender[1])
]
    gender_players_count = len(gender_df.groupby("SN"))
    return gender_players_count

# Create Lists to store Gender Demographics related data
gender_count = [count_players_by_gender(val) for val in gender]

gender_percentage = [round(val/total_players*100, 2) for val in gender_count]

# Create DataFrame 
gender_demographics = pd.DataFrame({
    "Total Count":gender_count,
    "Percentage of Players":gender_percentage},
    index = gender)

gender_demographics["Percentage of Players"] = gender_demographics["Percentage of Players"].map("{}%".format)
gender_demographics

,Total Count,Percentage of Players
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%


### Purchasing Analysis (Gender)
- Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender
- Create a summary data frame to hold the results
- Optional: give the displayed data cleaner formatting
- Display the summary data frame

In [415]:
# The below each broken by gender
# Define function to calculate count by gender
def count_by_gender(val):
    purchase_count_by_gender = df[["Gender", "Purchase ID"]].groupby(["Gender"]).count()
    return purchase_count_by_gender['Purchase ID'][val]

# Purchase Count
purchase_count = [count_by_gender(val) for val in gender]

# Average Purchase Price
purchase_price_by_gender = df[["Gender", "Price"]].groupby(["Gender"]).sum()

average_purchase = [
    round(purchase_price_by_gender['Price'][val]/purchase_count[gender.index(val)],2) for val in gender]

# Total Purchase Value
total_purchase = purchase_price_by_gender["Price"]

# Average Purchase Total per Person by Gender
# At first apply groupby by both columns - Gender and SD to find out the total purchase sum by each player
purchase_per_person = df[["Gender", "SN", "Price"]].groupby(["Gender", "SN"]).sum()

# Then use group by only by Gender to find the average purshase total per person by gender
average_purchase_per_person = round(purchase_per_person.groupby("Gender").mean(), 2)

avg_total = average_purchase_per_person["Price"]

# Create summary DataFrame
purchasing_analysis = pd.DataFrame({
    "Purchase Count": purchase_count,
    "Average Purchase Price": average_purchase,
    "Total Purchase Value": total_purchase,
    "Avg Total Purchase per Person": avg_total},
    index = gender
)
# Format table
purchasing_analysis["Average Purchase Price"] = purchasing_analysis["Average Purchase Price"].map("${:.2f}".format)
purchasing_analysis["Total Purchase Value"] = purchasing_analysis["Total Purchase Value"].map("${:.2f}".format)
purchasing_analysis["Avg Total Purchase per Person"] = purchasing_analysis["Avg Total Purchase per Person"].map("${:.2f}".format)

purchasing_analysis

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


### Age Demographics
- Establish bins for ages
- Categorize the existing players using the age bins. Hint: use pd.cut()
- Calculate the numbers and percentages by age group
- Create a summary data frame to hold the results
- Optional: round the percentage column to two decimal points
- Display Age Demographics Table

In [416]:
# The below each broken into bins of 4 years (i.e. &lt;10, 10-14, 15-19, etc.)
# Define min and max age of players to use it to form bins
min_age = df["Age"].min()
max_age = df["Age"].max()

# Create bins to categorize players by age, set the last bin[8] explicitly to the max age
bins = [x for x in range(min_age-3, max_age, 5)]
bins[8] = max_age

# Create bins labels
bins_labels = ["<10", "10-14", "15-19", "20-24", "25-29",
               "30-34)", "35-39", "40+"]

# Create DataFrame from the df with sorted values by Age to add the binning to
all_players_by_age_bins = df.sort_values(by = "Age")

# Add Column Age Group to categorize players by Age
all_players_by_age_bins["Age Group"] = pd.cut(all_players_by_age_bins["Age"], bins, labels = bins_labels)

# Change the values from categorical to string not to have NaN values while using groupby
all_players_by_age_bins["Age Group"] = all_players_by_age_bins["Age Group"].astype("str")

# Group by Age Group and SN to find unique number of players in each Age group
unique_players_by_age_bins = all_players_by_age_bins[["Age Group", "SN"]].groupby(
    ["Age Group", "SN"], sort = False)["SN"].count()

# Calculate the number of unique players in each age group
total_players_by_age_bins = unique_players_by_age_bins.groupby("Age Group").count()

# Create a list that holds percentage of players in total number of players by age group 
percentage_players_by_age = [f"{round(x/total_players*100,2)}%" for x in total_players_by_age_bins]

# Create a summary DataFrame
age_demographics = pd.DataFrame({
    "Total Count":total_players_by_age_bins,
    "Percentage of Players":percentage_players_by_age},
    index = bins_labels)
age_demographics

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34),52,9.03%
35-39,31,5.38%
40+,12,2.08%


### Purchasing Analysis (Age)
- Bin the purchase_data data frame by age
- Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below
- Create a summary data frame to hold the results
- Optional: give the displayed data cleaner formatting
- Display the summary data frame

In [417]:
# Purchase Count
purchase_count_by_age_bins = all_players_by_age_bins[["Age Group", "Purchase ID"]].groupby(
    ["Age Group"], sort = False)["Purchase ID"].count()

# Average Purchase Price
average_price_by_age_bins = round(all_players_by_age_bins[["Age Group", "Price"]].groupby(
    "Age Group", sort = False)["Price"].mean(), 2)

# Total Purchase Value
total_value_by_age_bins = round(all_players_by_age_bins[["Age Group", "Price"]].groupby(
    "Age Group", sort = False)["Price"].sum(), 2)

# Average Purchase Total per Person by Age Group
total_by_age_bins = all_players_by_age_bins.groupby(
    ["SN", "Age Group"], sort = False).sum()
average_total_by_age_bins = round(total_by_age_bins.groupby(
"Age Group", sort = False)["Price"].mean(),2)

# Summary DataFrame for Purchasing Analysis (Age)
purchasing_analysis_by_age = pd.DataFrame({
    "Purchase Count": purchase_count_by_age_bins,
    "Average Purchase Price": average_price_by_age_bins,
    "Total Purchase Value": total_value_by_age_bins,
    "Avg Total Purchase per Person": average_total_by_age_bins},
index = bins_labels)

# Format summary table
purchasing_analysis_by_age["Average Purchase Price"] = purchasing_analysis_by_age["Average Purchase Price"].map("${:.2f}".format)
purchasing_analysis_by_age["Total Purchase Value"] = purchasing_analysis_by_age["Total Purchase Value"].map("${:.2f}".format)
purchasing_analysis_by_age["Avg Total Purchase per Person"] = purchasing_analysis_by_age["Avg Total Purchase per Person"].map("${:.2f}".format)

purchasing_analysis_by_age

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34),73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


### Top Spenders
- Run basic calculations to obtain the results in the table below
- Create a summary data frame to hold the results
- Sort the total purchase value column in descending order
- Optional: give the displayed data cleaner formatting
- Display a preview of the summary data frame

In [418]:
# Identify the top 5 spenders in the game by total purchase value, then list (in a table):
# SN
# Purchase Count
# Average Purchase Price
# Total Purchase Value

# Find Purchase count and total purchase price
top_spenders = df.groupby("SN").agg({
    "Purchase ID": "count",
    "Price": "sum"}).sort_values(by=["Price"], ascending=False)

# Find Average Purchase Price
top_spenders["Average Purchase Price"] = round(df.groupby("SN")["Price"].mean(), 2)

top_spenders = top_spenders.rename(columns = {
    "Price": "Total Purchase Value",
    "Purchase ID": "Purchase Count"
})

# Format table to have right columns order and include $ sign for purshases values
top_spenders_format = top_spenders[["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]
top_spenders_format["Average Purchase Price"]=top_spenders_format["Average Purchase Price"].map("${:.2f}".format)
top_spenders_format["Total Purchase Value"]=top_spenders_format["Total Purchase Value"].map("${:.2f}".format)

print("Top spenders:")
top_spenders_format.head()

Top spenders:


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


### Most Popular Items
- Retrieve the Item ID, Item Name, and Item Price columns
- Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value
- Create a summary data frame to hold the results
- Sort the purchase count column in descending order
- Optional: give the displayed data cleaner formatting
- Display a preview of the summary data frame

In [419]:
# Identify the 5 most popular items by purchase count, then list (in a table):
# Item ID
# Item Name
# Purchase Count
# Item Price
# Total Purchase Value

# Define Purchase count and average price for each item
top_items = df.groupby(["Item ID", "Item Name"]).agg({
    "Purchase ID": "count",
    "Price": "mean"
})

# Find the total purchase value for each item
top_items["Total Purchase Value"] = df.groupby(["Item ID", "Item Name"])["Price"].sum()

# Create summary DataFrame sorted by Purchase Count to identify the most popular items
top_items = top_items.rename(columns = {"Purchase ID": "Purchase Count","Price": "Item Price"})
top_items = top_items.sort_values(by = ["Purchase Count"], ascending = False)

# Format table
# Create new DataFrame - top_items_format as we will use top_items table later 
top_items_format = top_items[["Purchase Count", "Item Price", "Total Purchase Value"]]
top_items_format["Total Purchase Value"] = top_items_format["Total Purchase Value"].map("${:.2f}".format)
top_items_format["Item Price"] = top_items_format["Item Price"].map("${:.2f}".format)

print("The most popular items:")
top_items_format.head()

The most popular items:


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


### Most Profitable Items
- Sort the above table by total purchase value in descending order
- Optional: give the displayed data cleaner formatting
- Display a preview of the data frame

In [420]:
# Identify the 5 most profitable items by total purchase value, then list (in a table):
# Item ID
# Item Name
# Purchase Count
# Item Price
# Total Purchase Value

# To find the most profitable items sort top_items DataFrame by Total Purchase Value
profit_items = top_items.sort_values(by = ["Total Purchase Value"], ascending=False)

# Format table
profit_items["Total Purchase Value"] = profit_items["Total Purchase Value"].map("${:.2f}".format)
profit_items["Item Price"] = profit_items["Item Price"].map("${:.2f}".format)

print("The most profitable items:")
profit_items.head(20)

The most profitable items:


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
59,"Lightning, Etcher of the King",8,$4.23,$33.84
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
78,"Glimmer, Ender of the Moon",7,$4.40,$30.80
72,Winter's Bite,8,$3.77,$30.16


### Analysis of the fantasy game Heroes of Pymoli
- Players of Male gender constitute the majority of all players: 84.03%.
- Purchases made by Male gender players make the majority of the total revenue: 1967.64 USD out of 2379.77 USD.
- The majority of all Players fall into 20-24 years age category: 44.79%.
- Players in 20-24 age category are also the most active purchasers: 1114.06 USD.
- The most profitable items (4.23 USD: 4.61 USD) are not the most expensive ones (4.93 USD : 4.99 USD).